In [9]:
from sqlalchemy.dialects.oracle.dictionary import all_objects

from models.TrackedObject import TrackedObject
from models.VideoAnalysis import VideoAnalysis
from utils.dynamics import EuclideanDistTracker, get_overlap_info
import utils.files as fil
import cv2
import os
import analysis.mask_rcnn as mask_rcnn
from analysis import yolo
%load_ext autoreload
%autoreload 2
isColab=False
if isColab:
    from google.colab.patches import cv2_imshow
    from google.colab import drive
from IPython.display import display, clear_output
import PIL.Image

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:

transform, COCO_INSTANCE_CATEGORY_NAMES, model, device= mask_rcnn.config()
model= yolo.config()

def process_clip(frame_filenames, case_folder):
    acc_detected=False
    tracker = EuclideanDistTracker()
    va=VideoAnalysis()
    # Get frame size dynamically from the first frame
    sample_frame = cv2.imread(frame_filenames[0])
    height, width = sample_frame.shape[:2]
    frame_size = (width, height)
    fps = 18

    # Save output to MP4 instead of AVI
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'avc1' for better compatibility
    out = cv2.VideoWriter(case_folder+"_output.mp4", fourcc, fps, frame_size)

    for frame_filename in frame_filenames:
        frame = cv2.imread(frame_filename)
        if frame is None:
            continue
        #result_frame,_,_objects = mask_rcnn.process_frame(model, frame, device, transform, COCO_INSTANCE_CATEGORY_NAMES, tracker, conf=0.5)
        result_frame, _objects = yolo.process_frame(model, frame, device, transform, COCO_INSTANCE_CATEGORY_NAMES, tracker, conf=0.5)
        for o in va.allObjects:
          o.is_present=False
          o.is_accident=False
        for obj in _objects:
          if obj.id>(len(va.allObjects)-1):
            va.allObjects.append(obj)
          else:
            o=va.allObjects[obj.id]
            o.id=obj.id
            o.is_present=obj.is_present
            o.x1=obj.x1
            o.x2=obj.x2
            o.y1=obj.y1
            o.y2=obj.y2
            o.centroid=obj.centroid
            o.overlaps=obj.overlaps
            o.past_centroids.append(obj.centroid)
            o.compute_speed()
            o.compute_acceleration()
            o.compute_vector()
            #print(str(o.id)+str(o.is_accident)+str(o.centroid))
        get_overlap_info(va.allObjects)
        for box in va.allObjects:
            box.check_accident()
            #cv2.putText(result_frame, str(box.acceleration),(box.x1, box.y1 - 10), cv2.FONT_HERSHEY_PLAIN, 1, (255, 0, 0), 2)
            if box.is_accident:
              acc_detected=True
              cv2.rectangle(result_frame, (box.x1, box.y1), (box.x2, box.y2), (0, 255, 255), 3)
        #print(frame_filename)
        out.write(result_frame)
        if isColab:
            #Convert to PIL format for proper display in Jupyter/Colab
            #Display only the current frame
            #clear_output(wait=True)
            result_frame_rgb = cv2.cvtColor(result_frame, cv2.COLOR_BGR2RGB)  # Convert BGR to RGB
            result_pil = PIL.Image.fromarray(result_frame_rgb)
            display(result_pil)
        else:
            cv2.imshow('res',result_frame)
            if cv2.waitKey(30) & 0xFF == ord('q'):
                  break
    out.release()
    cv2.destroyAllWindows()
    return acc_detected


Ultralytics 8.3.40  Python-3.12.7 torch-2.5.1+cpu CPU (12th Gen Intel Core(TM) i5-1235U)
Setup complete  (12 CPUs, 15.7 GB RAM, 393.8/454.3 GB disk)


In [11]:
if isColab:
    drive.mount('/content/drive/',force_remount=True)
    normal_folder = '/content/drive/MyDrive/data/normal/'
else:
    accident_folder = './data/accident/'
    normal_folder = './data/normal/'


folders = os.listdir(normal_folder)
print(folders)
accident_count, total_count=0,0
for case_folder in folders:
    frame_filenames= sorted(
        [f for f in os.listdir(normal_folder+case_folder) if f.endswith('.jpg')],
        key=fil.extract_number
    )

    frame_filenames = [os.path.join(normal_folder+case_folder, f) for f in frame_filenames]
    if process_clip(frame_filenames, case_folder) == True:
        accident_count+=1
    total_count+=1
print ("Total count: ",total_count)
print ("Accident count: ",accident_count)

['Normal_004.mp4', 'Normal_006.mp4', 'Normal_007.mp4', 'Normal_015.mp4', 'Normal_016.mp4']

0: 480x640 1 motorcycle, 1 truck, 384.8ms
Speed: 4.0ms preprocess, 384.8ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 409.9ms
Speed: 3.0ms preprocess, 409.9ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 truck, 423.2ms
Speed: 2.0ms preprocess, 423.2ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 1 truck, 394.8ms
Speed: 4.0ms preprocess, 394.8ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 415.7ms
Speed: 3.1ms preprocess, 415.7ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 truck, 366.7ms
Speed: 3.0ms preprocess, 366.7ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 1 truck, 366.9ms
Speed: 2.0ms preprocess, 366.9ms inference, 1.0ms postprocess per image at shape (1,

In [12]:
if isColab:
    # Process video frames
    drive.mount('/content/drive/',force_remount=True)
    folder = '/content/drive/MyDrive/Yolo/001102'
else:
    folder = './data/accident/'


folders = os.listdir(folder)
print(folders)
accident_count, total_count=0,0
for case_folder in folders:
    frame_filenames= sorted(
        [f for f in os.listdir(folder+case_folder) if f.endswith('.jpg')],
        key=fil.extract_number
    )

    frame_filenames = [os.path.join(folder+case_folder, f) for f in frame_filenames]
    if process_clip(frame_filenames, case_folder) == True:
        accident_count+=1
    total_count+=1
print ("Total count: ",total_count)
print ("Accident count: ",accident_count)

['01_Accident_001.mp4', '01_Accident_002.mp4', '01_Accident_004.mp4', '01_Accident_010.mp4', '01_Accident_011.mp4', '01_Accident_013.mp4']

0: 288x640 1 car, 1 motorcycle, 1 truck, 328.4ms
Speed: 3.0ms preprocess, 328.4ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 person, 1 car, 1 bus, 1 truck, 345.9ms
Speed: 4.0ms preprocess, 345.9ms inference, 2.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 person, 1 car, 1 bus, 1 truck, 302.3ms
Speed: 2.0ms preprocess, 302.3ms inference, 2.2ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 person, 1 car, 1 bus, 289.4ms
Speed: 2.0ms preprocess, 289.4ms inference, 1.0ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 car, 1 motorcycle, 1 bus, 288.5ms
Speed: 4.0ms preprocess, 288.5ms inference, 1.5ms postprocess per image at shape (1, 3, 288, 640)

0: 288x640 1 car, 1 motorcycle, 1 bus, 313.4ms
Speed: 2.0ms preprocess, 313.4ms inference, 1.0ms postprocess per image at 